In [1]:
%matplotlib inline

In [2]:
#import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
#from cv2 import VideoWriter, VideoWriter_fourcc
import pandas as pd
import torch
from scipy import signal, ndimage, spatial
from scipy.signal import correlate
from scipy.interpolate import CubicSpline
from scipy.ndimage import gaussian_filter
import math 

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_curve, auc

from scipy.interpolate import interp1d
import scipy.stats as stats
from scipy.stats import spearmanr
import seaborn as sns

Making the lookup tables needed for extracting kinematic features 

In [3]:
population = ['ALS', 'PLS', 'Kennedys', 'Post-stroke', 'Healthy_Control']

tasks = ['NSM_OPEN', 'NSM_SPREAD', 'NSM_BIGSMILE' , 'NSM_KISS', 'NSM_PUCKER']




# Paths 
base_path = '/media/deniz/easystore/facial_tracking_data_2021'

sub_folders_1 = '/02/TABLE'
sub_folders_2 = '/02/LANDMARKS/Deniz_SBR'


In [4]:
master_3Dsbr = []
master_table = []
master_rest = []
no_tables = []
subjects = []
tasks_list = []

for p in population:
    data_dir = os.path.join(base_path, p) 
    print()
    print('working on pop: ', p)
    
    if os.path.isdir(data_dir):
        for dirs in os.listdir(data_dir):
            table_dir = os.path.join(data_dir, dirs) + sub_folders_1
            sbr_dir = os.path.join(data_dir, dirs) + sub_folders_2
            
            present = []
            if os.path.exists(sbr_dir):
                sbr_list = os.listdir(sbr_dir)
                
                # get a single working rest file 
                rest = 0
                # GET THE REST FILE
                for f in sbr_list:
                    if ('RST_REST' in f) and (f.endswith('3D.csv')):
                        rest_file = os.path.join(sbr_dir, f)
                        rest = 1
                        
                if rest != 0:
                # GET THE SBR OF TASK FILES
                    present = [os.path.join(sbr_dir, i) for i in sbr_list if any(j in i for j in tasks) and i.endswith('3D.csv')]
               
                # lets get the table files of tasks 
                    for b in present:

                        name = os.path.basename(b)
                        name = name[:-29]
                        sub_id = name.split("_")[0]

                        table_file = os.path.join(table_dir, name + '_video.Table')
                        
                        if 'OPEN' in b:
                            task = 'OPEN'
                        elif 'SPREAD' in b:
                            task = 'SPREAD'
                        elif 'SMILE' in b:
                            task = 'BIGSMILE'
                        elif 'KISS' in b or 'PUCKER' in b:
                            task = 'PUCKER'
                        else:
                            task = 'UNKNOWN'
                        
                        if os.path.exists(table_file):
                            print('TABLE FILE: ',table_file, 'EXIST')
                            master_3Dsbr.append(b)
                            master_table.append(table_file)
                            master_rest.append(rest_file)
                            subjects.append(sub_id)
                            tasks_list.append(task)
                        else:
                            print(table_file, " DOES NOT EXIST")
                            #print('start')
                            #print(len(present))
                            #present.remove(b)
                            #print(len(present))
                            #print('end')
                            no_tables.append(b)


                
                        print('the rest file exisits then for ', sub_id)
                    #print(len(present))
                    #master_3Dsbr=master_3Dsbr+ present
                    #master_rest=master_rest+ [rest_file] * len(present)
                else:
                    print(dirs, ' HAS NO REST FILE')
                print('The len of present is: ',len(present))
                print(present)
                print()
print(len(master_3Dsbr),len(master_table),len(master_rest),len(no_tables), len(subjects))


working on pop:  ALS
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/TABLE/AF01_02_20170509_NSM_OPEN_video.Table EXIST
the rest file exisits then for  AF01
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/TABLE/AF01_02_20170509_NSM_SPREAD_video.Table EXIST
the rest file exisits then for  AF01
The len of present is:  2
['/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR/AF01_02_20170509_NSM_OPEN_color_SBR_3D_landmarks3D.csv', '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR/AF01_02_20170509_NSM_SPREAD_color_SBR_3D_landmarks3D.csv']

AF02_SBK_20170517  HAS NO REST FILE
The len of present is:  0
[]

TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/ALS/AF03_SBK_20170526/02/TABLE/AF03_02_20170526_NSM_OPEN_video.Table EXIST
the rest file exisits then for  AF03
TABLE FILE:  /media/deniz/easystore/facial_tracking_da

TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/ALS/AF18_SBK_20190826/02/TABLE/AF18_02_20190826_NSM_BIGSMILE_video.Table EXIST
the rest file exisits then for  AF18
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/ALS/AF18_SBK_20190826/02/TABLE/AF18_02_20190826_NSM_OPEN_video.Table EXIST
the rest file exisits then for  AF18
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/ALS/AF18_SBK_20190826/02/TABLE/AF18_02_20190826_NSM_PUCKER_1_video.Table EXIST
the rest file exisits then for  AF18
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/ALS/AF18_SBK_20190826/02/TABLE/AF18_02_20190826_NSM_SPREAD_video.Table EXIST
the rest file exisits then for  AF18
The len of present is:  4
['/media/deniz/easystore/facial_tracking_data_2021/ALS/AF18_SBK_20190826/02/LANDMARKS/Deniz_SBR/AF18_02_20190826_NSM_BIGSMILE_color_SBR_3D_landmarks3D.csv', '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF18_SBK_20190826/02/LANDMARKS/Deniz_SBR/AF18_02_20190826

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/TABLE/OP01_02_20190510_NSM_BIGSMILE_video.Table  DOES NOT EXIST
the rest file exisits then for  OP01
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/TABLE/OP01_02_20190510_NSM_KISS_video.Table  DOES NOT EXIST
the rest file exisits then for  OP01
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/TABLE/OP01_02_20190510_NSM_OPEN_video.Table  DOES NOT EXIST
the rest file exisits then for  OP01
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/TABLE/OP01_02_20190510_NSM_SPREAD_video.Table  DOES NOT EXIST
the rest file exisits then for  OP01
The len of present is:  4
['/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/LANDMARKS/Deniz_SBR/OP01_02_20190510_NSM_BIGSMILE_color_SBR_3D_landmarks3D.csv', '/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP01_SBK_20190510/02/LAND

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP03_SBK_20190524/02/TABLE/OP03_02_20190524_NSM_BIGSMILE_video.Table  DOES NOT EXIST
the rest file exisits then for  OP03
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP03_SBK_20190524/02/TABLE/OP03_02_20190524_NSM_OPEN_video.Table  DOES NOT EXIST
the rest file exisits then for  OP03
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP03_SBK_20190524/02/TABLE/OP03_02_20190524_NSM_PUCKER_video.Table  DOES NOT EXIST
the rest file exisits then for  OP03
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP03_SBK_20190524/02/TABLE/OP03_02_20190524_NSM_SPREAD_video.Table  DOES NOT EXIST
the rest file exisits then for  OP03
The len of present is:  4
['/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP03_SBK_20190524/02/LANDMARKS/Deniz_SBR/OP03_02_20190524_NSM_BIGSMILE_color_SBR_3D_landmarks3D.csv', '/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP03_SBK_20190524/02/LA

TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP07_SBK_20190906/02/TABLE/OP07_02_20190906_NSM_BIGSMILE_video.Table EXIST
the rest file exisits then for  OP07
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP07_SBK_20190906/02/TABLE/OP07_02_20190906_NSM_OPEN_video.Table EXIST
the rest file exisits then for  OP07
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP07_SBK_20190906/02/TABLE/OP07_02_20190906_NSM_PUCKER_video.Table  DOES NOT EXIST
the rest file exisits then for  OP07
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP07_SBK_20190906/02/TABLE/OP07_02_20190906_NSM_SPREAD_video.Table EXIST
the rest file exisits then for  OP07
The len of present is:  4
['/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP07_SBK_20190906/02/LANDMARKS/Deniz_SBR/OP07_02_20190906_NSM_BIGSMILE_color_SBR_3D_landmarks3D.csv', '/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP07_SBK_20190

TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP10_SBK_20190913/02/TABLE/OP10_02_20190913_NSM_BIGSMILE_video.Table EXIST
the rest file exisits then for  OP10
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP10_SBK_20190913/02/TABLE/OP10_02_20190913_NSM_OPEN_video.Table EXIST
the rest file exisits then for  OP10
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP10_SBK_20190913/02/TABLE/OP10_02_20190913_NSM_PUCKER_video.Table  DOES NOT EXIST
the rest file exisits then for  OP10
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP10_SBK_20190913/02/TABLE/OP10_02_20190913_NSM_SPREAD_video.Table EXIST
the rest file exisits then for  OP10
The len of present is:  4
['/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP10_SBK_20190913/02/LANDMARKS/Deniz_SBR/OP10_02_20190913_NSM_BIGSMILE_color_SBR_3D_landmarks3D.csv', '/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP10_SBK_20190

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/TABLE/OP12_02_20191206_NSM_PUCKER_video.Table  DOES NOT EXIST
the rest file exisits then for  OP12
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/TABLE/OP12_02_20191206_NSM_SPREAD_video.Table  DOES NOT EXIST
the rest file exisits then for  OP12
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/TABLE/OP12_02_20191206_NSM_BIGSMILE_video.Table  DOES NOT EXIST
the rest file exisits then for  OP12
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/TABLE/OP12_02_20191206_NSM_OPEN_video.Table  DOES NOT EXIST
the rest file exisits then for  OP12
The len of present is:  4
['/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/LANDMARKS/Deniz_SBR/OP12_02_20191206_NSM_PUCKER_color_SBR_3D_landmarks3D.csv', '/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP12_SBK_20191206/02/LAND

TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S001_TRI_20170403/02/TABLE/S001_02_20170403_NSM_OPEN_video.Table EXIST
the rest file exisits then for  S001
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S001_TRI_20170403/02/TABLE/S001_02_20170403_NSM_SPREAD_video.Table EXIST
the rest file exisits then for  S001
The len of present is:  2
['/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S001_TRI_20170403/02/LANDMARKS/Deniz_SBR/S001_02_20170403_NSM_OPEN_color_SBR_3D_landmarks3D.csv', '/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S001_TRI_20170403/02/LANDMARKS/Deniz_SBR/S001_02_20170403_NSM_SPREAD_color_SBR_3D_landmarks3D.csv']

TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S002_TRI_20170707/02/TABLE/S002_02_20170707_NSM_OPEN_video.Table EXIST
the rest file exisits then for  S002
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Post-stroke/S002_TRI_20170707/02/TABLE/S

TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF01_SBK_20170515/02/TABLE/NF01_02_20170515_NSM_OPEN_video.Table EXIST
the rest file exisits then for  NF01
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF01_SBK_20170515/02/TABLE/NF01_02_20170515_NSM_SPREAD_video.Table EXIST
the rest file exisits then for  NF01
The len of present is:  2
['/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF01_SBK_20170515/02/LANDMARKS/Deniz_SBR/NF01_02_20170515_NSM_OPEN_color_SBR_3D_landmarks3D.csv', '/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF01_SBK_20170515/02/LANDMARKS/Deniz_SBR/NF01_02_20170515_NSM_SPREAD_color_SBR_3D_landmarks3D.csv']

TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF02_SBK_20170517/02/TABLE/NF02_02_20170517_NSM_OPEN_video.Table EXIST
the rest file exisits then for  NF02
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF02

TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/TABLE/NF16_02_20190613_NSM_PUCKER_video.Table EXIST
the rest file exisits then for  NF16
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/TABLE/NF16_02_20190613_NSM_SPREAD_video.Table EXIST
the rest file exisits then for  NF16
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/TABLE/NF16_02_20190613_NSM_BIGSMILE_video.Table EXIST
the rest file exisits then for  NF16
TABLE FILE:  /media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/TABLE/NF16_02_20190613_NSM_OPEN_video.Table EXIST
the rest file exisits then for  NF16
The len of present is:  4
['/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF16_TRI_20190613/02/LANDMARKS/Deniz_SBR/NF16_02_20190613_NSM_PUCKER_color_SBR_3D_landmarks3D.csv', '/media/deniz/easystore/facial_tracking_data_2021/Healt

In [5]:
for b in master_3Dsbr:
    name = os.path.basename(b)
    name = name[:-23]
    
    

In [6]:
a = '_SBR_3D_landmarks3D.csv'
print(len(a))

23


In [7]:
master_3Dsbr

['/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR/AF01_02_20170509_NSM_OPEN_color_SBR_3D_landmarks3D.csv',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR/AF01_02_20170509_NSM_SPREAD_color_SBR_3D_landmarks3D.csv',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF03_SBK_20170526/02/LANDMARKS/Deniz_SBR/AF03_02_20170526_NSM_OPEN_color_SBR_3D_landmarks3D.csv',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF03_SBK_20170526/02/LANDMARKS/Deniz_SBR/AF03_02_20170526_NSM_SPREAD_color_SBR_3D_landmarks3D.csv',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF04_SBK_20170529/02/LANDMARKS/Deniz_SBR/AF04_02_20170529_NSM_OPEN_color_SBR_3D_landmarks3D.csv',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF04_SBK_20170529/02/LANDMARKS/Deniz_SBR/AF04_02_20170529_NSM_SPREAD_color_SBR_3D_landmarks3D.csv',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF05_SBK_20170530/02/

In [8]:
# save the results in the df and then save the df as a .csv 


df_cols = ['subject','landmarks', 'landmarks_table', 'rest', 'task']
df = pd.DataFrame(columns=df_cols)
df['subject'] = subjects
df['landmarks'] = master_3Dsbr
df['landmarks_table'] = master_table
df['rest'] = master_rest
df['task'] = tasks_list

look_path = os.path.join(base_path, 'sbr_3D_lookups.csv')
df.to_csv(look_path)

In [9]:
df.head()

,subject,landmarks,landmarks_table,rest,task
0,AF01,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,OPEN
1,AF01,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,SPREAD
2,AF03,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,OPEN
3,AF03,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,SPREAD
4,AF04,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,OPEN


In [8]:
look_path

'/media/deniz/easystore/facial_tracking_data_2021/sbr_3D_lookups.csv'

# How much data do we have available?

# total count for each task
## number of rssdk and bag files
## number of 3D converted files
## number of table files 
## number of participants in each population 

In [10]:

population = ['ALS', 'PLS', 'Kennedys', 'Post-stroke', 'Healthy_Control']

tasks = ['BBP_NORMAL', 'NSM_OPEN', 'NSM_SPREAD', 'NSM_BIGSMILE' , 'RST_REST', 'NSM_KISS', 'NSM_PUCKER', 'DDK_PA']




# Paths 
base_path = '/media/deniz/easystore/facial_tracking_data_2021'

sub_folders_1 = '/02/RSSDK'
sub_folders_2 = '/02/TABLE'


In [11]:
master_rssdk = []
master_rssdk_color = []
master_rssdk_depth = []

rssdk_subjects = []
rssdk_tasks_list = []

master_table = []
no_tables = []

master_bag = []
bag_subjects = []
bag_tasks_list = []

master_rest = []



# Getting Table Files Counts 

In [12]:
master_table = []

table_dirs = []
table_sub = []
table_date = []
table_task = []

for p in population:
    data_dir = os.path.join(base_path, p) 
    print()
    print('working on pop: ', p)
    
    if os.path.isdir(data_dir):
        for dirs in os.listdir(data_dir):
            table_dir = os.path.join(data_dir, dirs) + sub_folders_2
               
            present = []
            if os.path.exists(table_dir):
                tbl_list = os.listdir(table_dir)
                #print(tbl_list)
               
                present = [os.path.join(table_dir, i) for i in tbl_list if i.endswith('video.Table') or i.endswith('color.Table') ]
                table_dirs += [dirs]*len(present)
                table_sub += [dirs.split("_")[0]]*len(present)
                table_date += [dirs.split("_")[-1]]*len(present)
                for r in present:
                    name = os.path.basename(r)
                    
                    if 'OPEN' in r:
                        task = 'OPEN'
                    elif 'SPREAD' in r:
                        task = 'SPREAD'
                    elif 'SMILE' in r:
                        task = 'BIGSMILE'
                    elif 'KISS' in r or 'PUCKER' in r:
                        task = 'PUCKER'
                    elif 'BBP' in r:
                        task = 'BBP'
                    elif 'PA' in r and not 'KA' in r:
                        task = "PA"
                    elif 'REST' in r:
                        task = 'REST'
                    else:
                        task = 'NA'
                    table_task.append(task)
            master_table +=present      

print('Len master table count is ',len(master_table))

print('Len table dir count is ',len(table_dirs))


working on pop:  ALS

working on pop:  PLS

working on pop:  Kennedys

working on pop:  Post-stroke

working on pop:  Healthy_Control
Len master table count is  535
Len table dir count is  535


In [13]:
header = ['subID','Date','task','directory','table_path']
table_df = pd.DataFrame(columns=header)
table_df['directory'] = table_dirs
table_df['table_path'] = master_table
table_df['subID'] = table_sub
table_df['Date'] = table_date
table_df['task'] = table_task
print(len(table_df))
table_df.head()

535


,subID,Date,task,directory,table_path
0,AF01,20170509,NA,AF01_SBK_20170509,/media/deniz/easystore/facial_tracking_data_20...
1,AF01,20170509,PUCKER,AF01_SBK_20170509,/media/deniz/easystore/facial_tracking_data_20...
2,AF01,20170509,OPEN,AF01_SBK_20170509,/media/deniz/easystore/facial_tracking_data_20...
3,AF01,20170509,SPREAD,AF01_SBK_20170509,/media/deniz/easystore/facial_tracking_data_20...
4,AF02,20170517,NA,AF02_SBK_20170517,/media/deniz/easystore/facial_tracking_data_20...


In [14]:
table_df['table_path'][0]

'/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/TABLE/AF01_02_20170509_NSM_BLOW_video.Table'

# Getting RSSDK and Processed AVI Counts 

In [15]:
master_color = []
master_depth = []
master_rssdk = []
rssdk_sub = []
rssdk_date = []
avi_sub = []
avi_date = []
rssdk_task = []
avi_task = []


for p in population:
    data_dir = os.path.join(base_path, p) 
    print('working on pop: ', p)
    
    if os.path.isdir(data_dir):
        for dirs in os.listdir(data_dir):
            raw_dir = os.path.join(data_dir, dirs) + sub_folders_1
           
            if os.path.exists(raw_dir):
                rawdata_list = os.listdir(raw_dir)
                raw = [os.path.join(raw_dir, i) for i in rawdata_list if any(j in i for j in tasks) and i.endswith('.rssdk')and ('TA' not in i)]
                for r in raw:
                    name = os.path.basename(r)
                    rssdk_sub += [name.split("_")[0]]
                    rssdk_date += [name.split("_")[2]]
                    if 'OPEN' in r:
                        task = 'OPEN'
                    elif 'SPREAD' in r:
                        task = 'SPREAD'
                    elif 'SMILE' in r:
                        task = 'BIGSMILE'
                    elif 'KISS' in r or 'PUCKER' in r:
                        task = 'PUCKER'
                    elif 'BBP' in r:
                        task = 'BBP'
                    elif 'PA' in r and not 'KA' in r:
                        task = "PA"
                    elif 'REST' in r:
                        task = 'REST'
                    else:
                        task = 'NA'
                    rssdk_task.append(task)
                    
                present = [os.path.join(raw_dir, i) for i in rawdata_list if any(j in i for j in tasks) and i.endswith('depth.avi') and ('TA' not in i)]
                for b in present: 
                    name = b[:-9]
                    color =  name + 'color.avi'
                    if os.path.exists(color):
                        #color_file = os.path.join(raw_dir, name)
                        master_color.append(color)
                        avi_sub += [dirs.split("_")[0]]
                        avi_date += [dirs.split("_")[-1]]
                        
                    if 'OPEN' in b:
                        task = 'OPEN'
                    elif 'SPREAD' in b:
                        task = 'SPREAD'
                    elif 'SMILE' in b:
                        task = 'BIGSMILE'
                    elif 'KISS' in b or 'PUCKER' in b:
                        task = 'PUCKER'
                    elif 'BBP' in b:
                        task = 'BBP'
                    elif 'PA' in b and not 'KA' in b:
                        task = "PA"
                    elif 'REST' in b:
                        task = 'REST'
                    else:
                        task = 'NA'
                    avi_task.append(task)
                        
                    #name_2 =  os.path.basename(name)
                    #rssd = os.path.join(sbr_dir, name_2 + 'color_SBR_3D_landmarks.csv')
                    #master_pixels.append(pixels)
                master_depth=master_depth+ present
                master_rssdk +=raw
                #master_output=master_output+ [sbr_dir] * len(present)

print(len(master_depth),len(master_color),len(master_rssdk),len(avi_sub),len(avi_date), len(rssdk_sub),len(rssdk_date))

working on pop:  ALS
working on pop:  PLS
working on pop:  Kennedys
working on pop:  Post-stroke
working on pop:  Healthy_Control
132 132 220 132 132 220 220


In [16]:
header = ['subID','Date','task','rssdk_path']
rssdk_df = pd.DataFrame(columns=header)
rssdk_df['task'] = rssdk_task
rssdk_df['rssdk_path'] = master_rssdk
rssdk_df['subID'] = rssdk_sub
rssdk_df['Date'] = rssdk_date
print(len(rssdk_df))
rssdk_df.head()

220


,subID,Date,task,rssdk_path
0,AF01,20170509,BBP,/media/deniz/easystore/facial_tracking_data_20...
1,AF01,20170509,PA,/media/deniz/easystore/facial_tracking_data_20...
2,AF01,20170509,PUCKER,/media/deniz/easystore/facial_tracking_data_20...
3,AF01,20170509,OPEN,/media/deniz/easystore/facial_tracking_data_20...
4,AF01,20170509,SPREAD,/media/deniz/easystore/facial_tracking_data_20...


In [17]:
header = ['subID','Date','task','depth_path','color_path']
avi_df = pd.DataFrame(columns=header)
avi_df['task'] = avi_task
avi_df['depth_path'] = master_depth
avi_df['color_path'] = master_color
avi_df['subID'] = avi_sub
avi_df['Date'] = avi_date
print(len(avi_df))
avi_df.head()

132


,subID,Date,task,depth_path,color_path
0,AF01,20170509,BBP,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...
1,AF01,20170509,OPEN,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...
2,AF01,20170509,SPREAD,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...
3,AF01,20170509,REST,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...
4,AF03,20170526,BBP,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...


In [18]:
# merge the raw rssdk df with processed depth and color avi dfs

rssdk_df_full = rssdk_df.merge(avi_df, how= 'left', left_on=['subID','Date', 'task'], right_on=['subID','Date', 'task'])
print(len(rssdk_df_full))
rssdk_df_full.head()

220


,subID,Date,task,rssdk_path,depth_path,color_path
0,AF01,20170509,BBP,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...
1,AF01,20170509,PA,/media/deniz/easystore/facial_tracking_data_20...,NaN,NaN
2,AF01,20170509,PUCKER,/media/deniz/easystore/facial_tracking_data_20...,NaN,NaN
3,AF01,20170509,OPEN,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...
4,AF01,20170509,SPREAD,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...


In [21]:
look_path_r = os.path.join(base_path, 'all_rssdk_files_for_clinical_score.csv')
print(look_path_r)
rssdk_df_full.to_csv(look_path_r)

/media/deniz/easystore/facial_tracking_data_2021/all_rssdk_files_for_clinical_score.csv


# Getting bag file and avi Counts 

In [108]:
master_bag = []
bag_sub = []
bag_date = []
bag_task = []

master_color = []
avi_sub = []
avi_date = []
avi_task = []


for p in population:
    data_dir = os.path.join(base_path, p) 
    print('working on pop: ', p)
    
    if os.path.isdir(data_dir):
        for dirs in os.listdir(data_dir):
            print(dirs)
            raw_dir = os.path.join(data_dir, dirs) + sub_folders_1
           
            if os.path.exists(raw_dir):
                rawdata_list = os.listdir(raw_dir)
                raw = [os.path.join(raw_dir, i) for i in rawdata_list if any(j in i for j in tasks) and i.endswith('.bag')and ('TA' not in i)]
                print(len(raw))
                for r in raw:
                    name = os.path.basename(r)
                    bag_sub += [name.split("_")[0]]
                    bag_date += [name.split("_")[2]]
                    if 'OPEN' in r:
                        task = 'OPEN'
                    elif 'SPREAD' in r:
                        task = 'SPREAD'
                    elif 'SMILE' in r:
                        task = 'BIGSMILE'
                    elif 'KISS' in r or 'PUCKER' in r:
                        task = 'PUCKER'
                    elif 'BBP' in r:
                        task = 'BBP'
                    elif 'PA' in r and not 'KA' in r:
                        task = "PA"
                    elif 'REST' in r:
                        task = 'REST'
                    else:
                        task = 'NA'
                    bag_task.append(task)
                    
                    name = r[:-4]
                    color =  name + '_color.avi'
                    if os.path.exists(color):
                    
                #present = [os.path.join(raw_dir, i) for i in rawdata_list if any(j in i for j in tasks) and i.endswith('color.avi') and ('TA' not in i)]
                #print(len(present))
                #for b in present: 
                    
                        #color_file = os.path.join(raw_dir, name)
                       
                        avi_sub += [dirs.split("_")[0]]
                        avi_date += [dirs.split("_")[-1]]

                        
                        avi_task.append(task)
                        master_color.append(color)

                    #name_2 =  os.path.basename(name)
                    #rssd = os.path.join(sbr_dir, name_2 + 'color_SBR_3D_landmarks.csv')
                    #master_pixels.append(pixels)
                
                master_bag +=raw
                #master_output=master_output+ [sbr_dir] * len(present)

print(len(master_bag),len(master_color),len(avi_sub),len(avi_date), len(bag_sub),len(bag_date))

working on pop:  ALS
AF01_SBK_20170509
0
AF02_SBK_20170517
0
AF03_SBK_20170526
0
AF04_SBK_20170529
0
AF05_SBK_20170530
0
AF06_SBK_20170530
0
AF07_SBK_20170530
0
AF08_SBK_20170606
0
AF09_SBK_20170613
0
AF10_SBK_20170613
0
AF11_SBK_20170928
0
AF11_SBK_20200211
AF12_SBK_20180731
5
AF13_SBK_20190110
13
AF13_SBK_20190430
4
AF14_SBK_20190129
9
AF15_SBK_20190402
0
AF16_SBK_20190403
9
AF17_SBK_20190416
9
AF18_SBK_20190826
10
AF19_SBK_20190911
8
April_2021_Unprocessed_bag_files_ALS.csv
April_2021_Unprocessed_rssdk_files_ALS.csv
ASH1_SBK_20190409
4
ASH1_SBK_20190514
0
ASH2_SBK_20190730
0
working on pop:  PLS
April_2021_Unprocessed_bag_files_PLS.csv
P002_SBK_20190108
9
P003_SBK_20190123
9
P004_SBK_20190226
9
working on pop:  Kennedys
K001_SBK_20180420
11
K001_SBK_20190621
9
K003_SBK_20190809
9
working on pop:  Post-stroke
OP07_SBK_20191018
9
OP15_SBK_20200218
6
April_2021_Unprocessed_bag_files_PS.csv
April_2021_Unprocessed_rssdk_files_PS.csv
OP01_SBK_20190301
8
OP01_SBK_20190308
2
OP01_SBK_201905

In [109]:
header = ['subID','Date','task','bag_path']
bag_df = pd.DataFrame(columns=header)
bag_df['task'] = bag_task
bag_df['bag_path'] = master_bag
bag_df['subID'] = bag_sub
bag_df['Date'] = bag_date
print(len(bag_df))
bag_df.head()

714


,subID,Date,task,bag_path
0,AF12,20180731,PUCKER,/media/deniz/easystore/facial_tracking_data_20...
1,AF12,20180731,OPEN,/media/deniz/easystore/facial_tracking_data_20...
2,AF12,20180731,SPREAD,/media/deniz/easystore/facial_tracking_data_20...
3,AF12,20180731,REST,/media/deniz/easystore/facial_tracking_data_20...
4,AF12,20180731,REST,/media/deniz/easystore/facial_tracking_data_20...


In [140]:
print(len(bag_df))

714


In [142]:
look_path_bag = os.path.join(base_path, 'all_bag_files_for_clinical_score.csv')
print(look_path_bag)
bag_df.to_csv(look_path_bag)

/media/deniz/easystore/facial_tracking_data_2021/all_bag_files_for_clinical_score.csv


In [110]:
bag_df['bag_path'][3]

'/media/deniz/easystore/facial_tracking_data_2021/ALS/AF12_SBK_20180731/02/RSSDK/AF12_02_20180731_RST_REST.bag'

In [111]:
header = ['subID','Date','task','color_path']
avi_df = pd.DataFrame(columns=header)
avi_df['task'] = avi_task

avi_df['color_path'] = master_color
avi_df['subID'] = avi_sub
avi_df['Date'] = avi_date
print(len(avi_df))
avi_df.head()

655


,subID,Date,task,color_path
0,AF12,20180731,PUCKER,/media/deniz/easystore/facial_tracking_data_20...
1,AF12,20180731,OPEN,/media/deniz/easystore/facial_tracking_data_20...
2,AF12,20180731,SPREAD,/media/deniz/easystore/facial_tracking_data_20...
3,AF12,20180731,REST,/media/deniz/easystore/facial_tracking_data_20...
4,AF12,20180731,REST,/media/deniz/easystore/facial_tracking_data_20...


In [112]:
# merge the raw rssdk df with processed depth and color avi dfs

bag_df_full = bag_df.merge(avi_df, how= 'left', left_on=['subID','Date', 'task'], right_on=['subID','Date', 'task'])
print(len(bag_df_full))
bag_df_full.head()

1148


,subID,Date,task,bag_path,color_path
0,AF12,20180731,PUCKER,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...
1,AF12,20180731,OPEN,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...
2,AF12,20180731,SPREAD,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...
3,AF12,20180731,REST,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...
4,AF12,20180731,REST,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...


# PUTTING BAG, RSSDK, and Table files all in one DF

In [134]:
ids = ['A', 'O', 'S', 'N']
def pop_labeler(row):
    for i in ids:
        if i in row:
            if i == "A":
                return 'ALS'
            elif i == 'O' or i == "S":
                return 'Stroke'
            else:
                return 'HC'

In [113]:
print(len(rssdk_df_full))

rssdk_table_df = rssdk_df_full.merge(table_df, how= 'left', left_on=['subID','Date', 'task'], right_on=['subID','Date', 'task'])
print(len(rssdk_df_full))

rssdk_df_full.head()

231
231


,subID,Date,task,rssdk_path,depth_path,color_path,directory,table_path
0,AF01,20170509,BBP,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,NaN,NaN
1,AF01,20170509,PA,/media/deniz/easystore/facial_tracking_data_20...,NaN,NaN,NaN,NaN
2,AF01,20170509,PUCKER,/media/deniz/easystore/facial_tracking_data_20...,NaN,NaN,AF01_SBK_20170509,/media/deniz/easystore/facial_tracking_data_20...
3,AF01,20170509,OPEN,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,AF01_SBK_20170509,/media/deniz/easystore/facial_tracking_data_20...
4,AF01,20170509,SPREAD,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,AF01_SBK_20170509,/media/deniz/easystore/facial_tracking_data_20...


# NUmbers are here for RSSDK

In [129]:
print('The number of missing table files are ',rssdk_df_full[['table_path']].isna().sum())
print()
print('The number of missing depth files are ',rssdk_df_full[['depth_path']].isna().sum())
print()
print('The number of missing color files are ',rssdk_df_full[['color_path']].isna().sum())


The number of missing table files are  table_path    95
dtype: int64

The number of missing depth files are  depth_path    91
dtype: int64

The number of missing color files are  color_path    91
dtype: int64


In [131]:
print(rssdk_df_full['subID'].unique())

['AF01' 'AF02' 'AF03' 'AF04' 'AF05' 'AF06' 'AF07' 'AF08' 'AF09' 'AF10'
 'AF11' 'S001' 'S002' 'S003' 'S004' 'S005' 'S006' 'S007' 'S008' 'S009'
 'S010' 'S011' 'S012' 'S013' 'NF00' 'NF01' 'NF02' 'NF03' 'NF04' 'NF05'
 'NF06' 'NF07' 'NF08' 'NF09' 'NF10']


In [135]:
rssdk_df_full['population'] = [pop_labeler(x) for x in rssdk_df_full['subID']]

In [137]:
rssdk_df_full.groupby(['population', 'task']).count()

subID  Date  rssdk_path  depth_path  color_path  directory  \
population task                                                                 
ALS        BBP        11    11          11          10          10          2   
           OPEN       11    11          11          10          10         11   
           PA         11    11          11           0           0          0   
           PUCKER     11    11          11           0           0         11   
           REST       11    11          11          10          10          0   
           SPREAD     11    11          11          10          10         11   
HC         BBP        12    12          12          11          11          0   
           OPEN       12    12          12          11          11         11   
           PA         11    11          11           0           0         10   
           PUCKER     11    11          11           0           0         10   
           REST       14    14          14          13          13          0   
           SPREAD     12    12          12          11          11         11   
Stroke     BBP        12    12          12          12          12          0   
           OPEN       13    13          13          12          12         12   
           PA         12    12          12           0           0         12   
           PUCKER     24    24          24           0           0         23   
           REST       19    19          19          18          18          0   
           SPREAD     13    13          13          12          12         12   

                   table_path  
population task                
ALS        BBP              2  
           OPEN            11  
           PA               0  
           PUCKER          11  
           REST             0  
           SPREAD          11  
HC         BBP              0  
           OPEN            11  
           PA              10  
           PUCKER          10  
           REST             0  
           SPREAD          11  
Stroke     BBP              0  
           OPEN            12  
           PA              12  
           PUCKER          23  
           REST             0  
           SPREAD          12

In [116]:
# starting bag file analysis 

print(len(bag_df_full))

bag_table_df = bag_df_full.merge(table_df, how= 'left', left_on=['subID','Date', 'task'], right_on=['subID','Date', 'task'])
print(len(bag_table_df))

bag_table_df.head()

1148
1178


,subID,Date,task,bag_path,color_path,directory,table_path
0,AF12,20180731,PUCKER,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,NaN,NaN
1,AF12,20180731,OPEN,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,NaN,NaN
2,AF12,20180731,SPREAD,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,NaN,NaN
3,AF12,20180731,REST,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,NaN,NaN
4,AF12,20180731,REST,/media/deniz/easystore/facial_tracking_data_20...,/media/deniz/easystore/facial_tracking_data_20...,NaN,NaN


In [136]:
bag_table_df['population'] = [pop_labeler(x) for x in bag_table_df['subID']]

In [139]:

print('need parsing',bag_table_df[['table_path']].isna().sum())

need parsing table_path    490
dtype: int64


# NUmbers are here FOR BAG

In [138]:
bag_table_df.groupby(['population', 'task']).count()

subID  Date  bag_path  color_path  directory  table_path
population task                                                              
ALS        BBP           7     7         7           6          4           4
           BIGSMILE     12    12        12          10         10          10
           OPEN         11    11        11           9          6           6
           PA            7     7         7           6          0           0
           PUCKER        7     7         7           6          5           5
           REST         63    63        63          62         34          34
           SPREAD       10    10        10           8          6           6
HC         BBP          28    28        28          27          6           6
           BIGSMILE     35    35        35          33         27          27
           OPEN         35    35        35          33         28          28
           PA           24    24        24          15          0           0
           PUCKER       24    24        24          15          9           9
           REST        140   140       140         140         91          91
           SPREAD       40    40        40          39         33          33
Stroke     BBP          42    42        42          39         10          10
           BIGSMILE     46    46        46          41         30          30
           OPEN         44    44        44          40         32          32
           PA           64    64        64          59          0           0
           PUCKER       42    42        42          39         26          26
           REST        352   352       352         352        285         285
           SPREAD       47    47        47          43         33          33

# general testing

In [35]:
for p in population:
    data_dir = os.path.join(base_path, p) 
    print()
    print('working on pop: ', p)
    
    if os.path.isdir(data_dir):
        for dirs in os.listdir(data_dir):
            raw_dir = os.path.join(data_dir, dirs) + sub_folders_1
            
            present = []
            if os.path.exists(sbr_dir):
                sbr_list = os.listdir(sbr_dir)
                
                
                depth_present = [os.path.join(sbr_dir, i) for i in sbr_list if any(j in i for j in tasks) and i.endswith('depth.avi')]
                depth_present = [os.path.join(sbr_dir, i) for i in sbr_list if any(j in i for j in tasks) and i.endswith('color.avi')]
                # lets get the table files of tasks 
                    for b in present:

                        name = os.path.basename(b)
                        name = name[:-29]
                        sub_id = name.split("_")[0]

                        table_file = os.path.join(table_dir, name + '_video.Table')
                        
                        if 'OPEN' in b:
                            task = 'OPEN'
                        elif 'SPREAD' in b:
                            task = 'SPREAD'
                        elif 'SMILE' in b:
                            task = 'BIGSMILE'
                        elif 'KISS' in b or 'PUCKER' in b:
                            task = 'PUCKER'
                        elif 'BBP' in b:
                            task = 'BBP'
                        elif 'PA' in b and not 'KA' in b:
                            task = "PA"
                        else:
                            task = 'NA'
                        
                        if os.path.exists(table_file):
                            print('TABLE FILE: ',table_file, 'EXIST')
                            master_3Dsbr.append(b)
                            master_table.append(table_file)
                            master_rest.append(rest_file)
                            subjects.append(sub_id)
                            tasks_list.append(task)
                        else:
                            print(table_file, " DOES NOT EXIST")
                            #print('start')
                            #print(len(present))
                            #present.remove(b)
                            #print(len(present))
                            #print('end')
                            no_tables.append(b)


                
                        print('the rest file exisits then for ', sub_id)
                    #print(len(present))
                    #master_3Dsbr=master_3Dsbr+ present
                    #master_rest=master_rest+ [rest_file] * len(present)
                else:
                    print(dirs, ' HAS NO REST FILE')
                print('The len of present is: ',len(present))
                print(present)
                print()
print(len(master_3Dsbr),len(master_table),len(master_rest),len(no_tables), len(subjects))

['AF01_SBK_20170509',
 'AF01_SBK_20170509',
 'AF01_SBK_20170509',
 'AF01_SBK_20170509',
 'AF02_SBK_20170517',
 'AF02_SBK_20170517',
 'AF02_SBK_20170517',
 'AF02_SBK_20170517',
 'AF03_SBK_20170526',
 'AF03_SBK_20170526',
 'AF03_SBK_20170526',
 'AF03_SBK_20170526',
 'AF04_SBK_20170529',
 'AF04_SBK_20170529',
 'AF04_SBK_20170529',
 'AF04_SBK_20170529',
 'AF05_SBK_20170530',
 'AF05_SBK_20170530',
 'AF05_SBK_20170530',
 'AF05_SBK_20170530',
 'AF05_SBK_20170530',
 'AF06_SBK_20170530',
 'AF06_SBK_20170530',
 'AF06_SBK_20170530',
 'AF06_SBK_20170530',
 'AF07_SBK_20170530',
 'AF07_SBK_20170530',
 'AF07_SBK_20170530',
 'AF07_SBK_20170530',
 'AF08_SBK_20170606',
 'AF08_SBK_20170606',
 'AF08_SBK_20170606',
 'AF08_SBK_20170606',
 'AF09_SBK_20170613',
 'AF09_SBK_20170613',
 'AF09_SBK_20170613',
 'AF09_SBK_20170613',
 'AF09_SBK_20170613',
 'AF10_SBK_20170613',
 'AF10_SBK_20170613',
 'AF10_SBK_20170613',
 'AF10_SBK_20170613',
 'AF11_SBK_20170928',
 'AF11_SBK_20170928',
 'AF11_SBK_20170928',
 'AF11_SBK

In [27]:
for p in population:
    data_dir = os.path.join(base_path, p) 
    print()
    print('working on pop: ', p)
    
    if os.path.isdir(data_dir):
        for dirs in os.listdir(data_dir):
            table_dir = os.path.join(data_dir, dirs) + sub_folders_1
            sbr_dir = os.path.join(data_dir, dirs) + sub_folders_2
            
            present = []
            if os.path.exists(sbr_dir):
                sbr_list = os.listdir(sbr_dir)
                
                # get a single working rest file 
                rest = 0
                # GET THE REST FILE
                for f in sbr_list:
                    if ('RST_REST' in f) and (f.endswith('3D.csv')):
                        rest_file = os.path.join(sbr_dir, f)
                        rest = 1
                        
                if rest != 0:
                # GET THE SBR OF TASK FILES
                    present = [os.path.join(sbr_dir, i) for i in sbr_list if any(j in i for j in tasks) and i.endswith('3D.csv')]
               
                # lets get the table files of tasks 
                    for b in present:

                        name = os.path.basename(b)
                        name = name[:-29]
                        sub_id = name.split("_")[0]

                        table_file = os.path.join(table_dir, name + '_video.Table')
                        
                        if 'OPEN' in b:
                            task = 'OPEN'
                        elif 'SPREAD' in b:
                            task = 'SPREAD'
                        elif 'SMILE' in b:
                            task = 'BIGSMILE'
                        elif 'KISS' in b or 'PUCKER' in b:
                            task = 'PUCKER'
                        elif 'BBP' in b:
                            task = 'BBP'
                        elif 'PA' in b and not 'KA' in b:
                            task = "PA"
                        else:
                            task = 'NA'
                        
                        if os.path.exists(table_file):
                            print('TABLE FILE: ',table_file, 'EXIST')
                            master_3Dsbr.append(b)
                            master_table.append(table_file)
                            master_rest.append(rest_file)
                            subjects.append(sub_id)
                            tasks_list.append(task)
                        else:
                            print(table_file, " DOES NOT EXIST")
                            #print('start')
                            #print(len(present))
                            #present.remove(b)
                            #print(len(present))
                            #print('end')
                            no_tables.append(b)


                
                        print('the rest file exisits then for ', sub_id)
                    #print(len(present))
                    #master_3Dsbr=master_3Dsbr+ present
                    #master_rest=master_rest+ [rest_file] * len(present)
                else:
                    print(dirs, ' HAS NO REST FILE')
                print('The len of present is: ',len(present))
                print(present)
                print()
print(len(master_3Dsbr),len(master_table),len(master_rest),len(no_tables), len(subjects))


working on pop:  ALS
AF01_SBK_20170509  HAS NO REST FILE
The len of present is:  0
[]

AF02_SBK_20170517  HAS NO REST FILE
The len of present is:  0
[]

AF03_SBK_20170526  HAS NO REST FILE
The len of present is:  0
[]

AF04_SBK_20170529  HAS NO REST FILE
The len of present is:  0
[]

AF05_SBK_20170530  HAS NO REST FILE
The len of present is:  0
[]

AF06_SBK_20170530  HAS NO REST FILE
The len of present is:  0
[]

AF07_SBK_20170530  HAS NO REST FILE
The len of present is:  0
[]

AF08_SBK_20170606  HAS NO REST FILE
The len of present is:  0
[]

AF09_SBK_20170613  HAS NO REST FILE
The len of present is:  0
[]

AF10_SBK_20170613  HAS NO REST FILE
The len of present is:  0
[]

AF11_SBK_20170928  HAS NO REST FILE
The len of present is:  0
[]

AF12_SBK_20180731  HAS NO REST FILE
The len of present is:  0
[]

AF13_SBK_20190110  HAS NO REST FILE
The len of present is:  0
[]

AF13_SBK_20190430  HAS NO REST FILE
The len of present is:  0
[]

AF14_SBK_20190129  HAS NO REST FILE
The len of present

In [ ]:
table_files = []

for p in population:
    data_dir = os.path.join(base_path, p) 
    print()
    print('working on pop: ', p)
    
    if os.path.isdir(data_dir):
        for dirs in os.listdir(data_dir):
            table_dir = os.path.join(data_dir, dirs) + sub_folders_1
            
            present = []
            if os.path.exists(table_dir):
                sbr_list = os.listdir(table_dir)
                
                    
                present = [os.path.join(table_dir, i) for i in sbr_list if i.endswith('video.Table') and ("RST" not in i) and ("BBP" not in i)]
               
            table_files += present
            
print(len(table_files))
table_files

OLD DATA

In [3]:
file = open(r"/Users/denizjafari/Documents/CODE/ClinicalScore/ClinicalScore/facial_tracking_data_all.txt","r") 

print(file.read())

./ALS/AF01_SBK_20170509/02/CLINICAL/AF01_050917_OME notes.pdf
./ALS/AF01_SBK_20170509/02/CLINICAL/AF01_050917_OME.pdf
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_NORMAL.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_KA.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_PA.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_PATAKA.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_TA.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_BLOW.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_KISS.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_OPEN.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_SPREAD.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_RST_REST.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_TNG_LATFAST.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_TNG_LATNORMAL.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_TNG_NOSE.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_TNG_OUT.rssdk
./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_TNG_PROTRUSION.rssdk
./ALS/AF01_SBK_20170509/02/RS

In [4]:
data = pd.read_csv("/Users/denizjafari/Documents/CODE/ClinicalScore/ClinicalScore/facial_tracking_data_all.txt",  header=None)

In [5]:
print(data.shape)
data

(5804, 1)


,0
0,./ALS/AF01_SBK_20170509/02/CLINICAL/AF01_05091...
1,./ALS/AF01_SBK_20170509/02/CLINICAL/AF01_05091...
2,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_N...
3,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_K...
4,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...
...,...
5799,./DATASET_updated_20191018.xlsx
5800,./Acoustic_DataAcquisitionLog.xlsx
5801,./content_testing.txt
5802,./._content_testing.txt


In [6]:
tasks = ["BBP_NORMAL" ,"DDK_PA" , "NSM_BIGSMILE" ,"NSM_OPEN" , "NSM_SPREAD" , "RST_REST" ,"NSM_KISS"]




def wanted_files(row):
       
    if '.rssdk' in os.path.basename(row) or '.bag' in os.path.basename(row):
        return os.path.basename(row)
    else:
        return None
        
def get_subject_id(row):
    
    return row.split('_')[0]        
        

def set_task(row):
    for t in tasks:
        if t in row:
            if t != 'DDK_PATAKA'
                return t
  

data.rename(columns = {0: 'file_path'}, inplace = True)


print(data.shape)
data['file_name'] = [wanted_files(x) for x in data['file_path']]

print(data.shape)
data = data[data['file_name'].notna()]
print(data.shape)
data['subject_id'] = [get_subject_id(x) for x in data['file_name']]
data['task'] = [set_task(x) for x in data['file_name']]
data = data[data['task'].notna()]


print(data.shape)
print(data.task.unique())
data

(5804, 1)
(5804, 2)
(4357, 2)
(1135, 4)
['BBP_NORMAL' 'DDK_PA' 'NSM_KISS' 'NSM_OPEN' 'NSM_SPREAD' 'RST_REST'
 'NSM_BIGSMILE']


<ipython-input-6-159ee699571d>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['subject_id'] = [get_subject_id(x) for x in data['file_name']]
<ipython-input-6-159ee699571d>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['task'] = [set_task(x) for x in data['file_name']]


,file_path,file_name,subject_id,task
2,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_N...,AF01_02_BBP_NORMAL.rssdk,AF01,BBP_NORMAL
4,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PA.rssdk,AF01,DDK_PA
5,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PATAKA.rssdk,AF01,DDK_PA
8,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_K...,AF01_02_NSM_KISS.rssdk,AF01,NSM_KISS
9,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_O...,AF01_02_NSM_OPEN.rssdk,AF01,NSM_OPEN
...,...,...,...,...
5776,./Post-Stroke/S013_TRI_20180302/02/RSSDK/S013_...,S013_02_RST_REST3.rssdk,S013,RST_REST
5795,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_KISS.rssdk,S004,NSM_KISS
5796,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_OPEN.rssdk,S004,NSM_OPEN
5797,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_SPREAD.rssdk,S004,NSM_SPREAD


In [7]:
# data that already has clinical score 

path = r'/Users/denizjafari/documents/CODE/ClinicalScore/NeuroFace_Dataset/ALS/SLP_Assessment_ALS.csv'
df_als = pd.read_csv(path)

df_als = df_als.drop('Unnamed: 8', axis=1)

#print(df_als.shape)
#print(df_als.columns)
#print(df_als['Subject ID'].unique())
#df_als.head()

# STROKE DATA
path = r'/Users/denizjafari/documents/CODE/ClinicalScore/NeuroFace_Dataset/Stroke/SLP_Assessment_PS.csv'
df_ps = pd.read_csv(path)

df_ps = df_ps.drop('Unnamed: 8', axis=1)

#print(df_ps.shape)
#print(df_ps.columns)
#print(df_ps['Subject ID'].unique())
#df_ps.head()

# HEALTHY CONTROL DATA
path = r'/Users/denizjafari/documents/CODE/ClinicalScore/NeuroFace_Dataset/Healthy controls/SLP_Assessment_HC.csv'
df_hc = pd.read_csv(path)

df_hc = df_hc.drop('Unnamed: 8', axis=1)
df_hc = df_hc.drop('Unnamed: 15', axis=1)
#print(df_hc.shape)
#print(df_hc.columns)
#print(df_hc['Subject ID'].unique())




slp_scores = pd.concat([df_als, df_ps, df_hc], ignore_index=True)
print(slp_scores.shape)
#slp_scores['File Name'] = [wanted_tasks_slp(x) for x in slp_scores['File Name']]

slp_scores['task'] = [set_task(x) for x in slp_scores['File Name']]

slp_scores = slp_scores[slp_scores['task'].notna()]

print(slp_scores.shape)

slp_scores

(261, 14)
(222, 15)


,File Name,Subject ID,Symmetry (SLP1),ROM (SLP1),Speed (SLP1),Variability (SLP1),Fatigue (SLP1),Tot (SLP1),Symmetry (SLP2),ROM (SLP2),Speed (SLP2),Variability (SLP2),Fatigue (SLP2),Tot (SLP2),task
0,A002_02_BBP_NORMAL_color.avi,A002,1,1,1,1,1,5,2,1,1,1,1,6,BBP_NORMAL
1,A002_02_DDK_PATAKA_color.avi,A002,1,1,1,1,1,5,2,2,1,2,2,9,DDK_PA
2,A002_02_DDK_PA_color.avi,A002,1,1,1,1,1,5,2,2,1,2,2,9,DDK_PA
4,A002_02_NSM_KISS_color.avi,A002,2,1,1,1,4,9,1,1,1,1,3,7,NSM_KISS
5,A002_02_NSM_OPEN_color.avi,A002,1,1,1,1,1,5,1,1,1,1,1,5,NSM_OPEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,N019_02_DDK_PA_color.avi,N019,2,1,1,1,1,6,1,1,1,1,1,5,DDK_PA
256,N019_02_NSM_BIGSMILE_color.avi,N019,1,2,2,1,1,7,2,2,2,2,1,9,NSM_BIGSMILE
258,N019_02_NSM_KISS_color.avi,N019,2,1,2,2,1,8,2,1,1,1,1,6,NSM_KISS
259,N019_02_NSM_OPEN_color.avi,N019,2,1,1,1,1,6,2,1,1,1,1,6,NSM_OPEN


In [8]:
print(slp_scores['Subject ID'].unique())
print(len(slp_scores['Subject ID'].unique()))

def id_label_change(sub_id):
    s = sub_id
    if 'N' in sub_id:         
        s = s.replace('N0', 'NF')
    elif 'A' in sub_id:
        s = s.replace('A0', 'AF')

    return s

slp_scores['Subject ID'] = [id_label_change(x) for x in slp_scores['Subject ID']]
slp_scores['task'] = [set_task(x) for x in slp_scores['File Name']]



print(len(slp_scores['Subject ID'].unique()))
slp_scores['Subject ID'].unique()

['A002' 'A006' 'A008' 'A009' 'A010' 'A011' 'A012' 'A014' 'A015' 'A016'
 'A017' 'OP01' 'OP02' 'OP03' 'S001' 'S002' 'S003' 'S005' 'S006' 'S007'
 'S008' 'S009' 'S011' 'S012' 'S013' 'N001' 'N002' 'N003' 'N004' 'N007'
 'N008' 'N010' 'N011' 'N012' 'N017' 'N019']
36
36


array(['AF02', 'AF06', 'AF08', 'AF09', 'AF10', 'AF11', 'AF12', 'AF14',
       'AF15', 'AF16', 'AF17', 'OP01', 'OP02', 'OP03', 'S001', 'S002',
       'S003', 'S005', 'S006', 'S007', 'S008', 'S009', 'S011', 'S012',
       'S013', 'NF01', 'NF02', 'NF03', 'NF04', 'NF07', 'NF08', 'NF10',
       'NF11', 'NF12', 'NF17', 'NF19'], dtype=object)

In [9]:
print(slp_scores.shape)
print(data.shape)
print(len(data['subject_id'].unique()))
data['subject_id'].unique()

(222, 15)
(1135, 4)
88


array(['AF01', 'AF02', 'AF03', 'AF04', 'AF05', 'AF06', 'AF07', 'AF09',
       'AF10', 'AF11', 'AF12', 'AF13', 'AF14', 'AF16', 'AF17', 'AF18',
       'AF19', 'ASH1', 'NF00', 'NF01', 'NF02', 'NF03', 'NF04', 'NF05',
       'NF06', 'NF07', 'NF08', 'NF09', 'NF10', 'NF11', 'NF12', 'NF13',
       'NF14', 'NF15', 'NF16', 'NF17', 'NF18', 'NF19', 'NF20', 'NF21',
       'NF22', 'NF23', 'NF24', 'NF25', 'NF26', 'NF27', '001', '002',
       'K001', 'K003', 'PD01', 'PD02', 'PD03', 'PD04', 'PD05', 'PD06',
       'PD07', 'PD08', 'P002', 'P003', 'P004', 'OP01', 'OP02', 'OP03',
       'OP04', 'OP05', 'OP06', 'OP07', 'OP08', 'OP09', 'OP10', 'OP11',
       'OP12', 'OP13', 'OP14', 'S001', 'S002', 'S003', 'S005', 'S006',
       'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S004'],
      dtype=object)

In [10]:
test = data.merge(slp_scores, how= 'left', left_on=['subject_id', 'task'], right_on=['Subject ID', 'task'])
print(test.shape)
test.head()

(1245, 18)


,file_path,file_name,subject_id,task,File Name,Subject ID,Symmetry (SLP1),ROM (SLP1),Speed (SLP1),Variability (SLP1),Fatigue (SLP1),Tot (SLP1),Symmetry (SLP2),ROM (SLP2),Speed (SLP2),Variability (SLP2),Fatigue (SLP2),Tot (SLP2)
0,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_N...,AF01_02_BBP_NORMAL.rssdk,AF01,BBP_NORMAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PA.rssdk,AF01,DDK_PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PATAKA.rssdk,AF01,DDK_PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_K...,AF01_02_NSM_KISS.rssdk,AF01,NSM_KISS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_O...,AF01_02_NSM_OPEN.rssdk,AF01,NSM_OPEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
test.columns

Index(['file_path', 'file_name', 'subject_id', 'task', 'File Name',
       'Subject ID', 'Symmetry (SLP1)', 'ROM (SLP1)', 'Speed (SLP1)',
       'Variability (SLP1)', 'Fatigue (SLP1)', 'Tot (SLP1)', 'Symmetry (SLP2)',
       'ROM (SLP2)', 'Speed (SLP2)', 'Variability (SLP2)', 'Fatigue (SLP2)',
       'Tot (SLP2)'],
      dtype='object')

In [12]:
len(set(data['subject_id']).intersection(set(slp_scores['Subject ID'])))
#set(data['subject_id']).intersection(set(slp_scores['Subject ID']))

34

In [13]:
test['Need_Clinical_Score'] = np.where(test['Tot (SLP1)'].isnull() , 1, 0)
#test['Need_Clinical_Score'].loc[test['Tot (SLP1)'] == None] = 1
#test['Need_Clinical_Score'].loc[test['Need_Clinical_Score'] != 1] = 0

test = test.drop(['File Name',
       'Subject ID', 'Symmetry (SLP1)', 'ROM (SLP1)', 'Speed (SLP1)',
       'Variability (SLP1)', 'Fatigue (SLP1)', 'Tot (SLP1)', 'Symmetry (SLP2)',
       'ROM (SLP2)', 'Speed (SLP2)', 'Variability (SLP2)', 'Fatigue (SLP2)',
       'Tot (SLP2)'], axis = 1)

test

,file_path,file_name,subject_id,task,Need_Clinical_Score
0,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_N...,AF01_02_BBP_NORMAL.rssdk,AF01,BBP_NORMAL,1
1,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PA.rssdk,AF01,DDK_PA,1
2,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PATAKA.rssdk,AF01,DDK_PA,1
3,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_K...,AF01_02_NSM_KISS.rssdk,AF01,NSM_KISS,1
4,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_O...,AF01_02_NSM_OPEN.rssdk,AF01,NSM_OPEN,1
...,...,...,...,...,...
1240,./Post-Stroke/S013_TRI_20180302/02/RSSDK/S013_...,S013_02_RST_REST3.rssdk,S013,RST_REST,1
1241,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_KISS.rssdk,S004,NSM_KISS,1
1242,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_OPEN.rssdk,S004,NSM_OPEN,1
1243,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_SPREAD.rssdk,S004,NSM_SPREAD,1


In [14]:
test.subject_id.unique()

array(['AF01', 'AF02', 'AF03', 'AF04', 'AF05', 'AF06', 'AF07', 'AF09',
       'AF10', 'AF11', 'AF12', 'AF13', 'AF14', 'AF16', 'AF17', 'AF18',
       'AF19', 'ASH1', 'NF00', 'NF01', 'NF02', 'NF03', 'NF04', 'NF05',
       'NF06', 'NF07', 'NF08', 'NF09', 'NF10', 'NF11', 'NF12', 'NF13',
       'NF14', 'NF15', 'NF16', 'NF17', 'NF18', 'NF19', 'NF20', 'NF21',
       'NF22', 'NF23', 'NF24', 'NF25', 'NF26', 'NF27', '001', '002',
       'K001', 'K003', 'PD01', 'PD02', 'PD03', 'PD04', 'PD05', 'PD06',
       'PD07', 'PD08', 'P002', 'P003', 'P004', 'OP01', 'OP02', 'OP03',
       'OP04', 'OP05', 'OP06', 'OP07', 'OP08', 'OP09', 'OP10', 'OP11',
       'OP12', 'OP13', 'OP14', 'S001', 'S002', 'S003', 'S005', 'S006',
       'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S004'],
      dtype=object)

### Need Parsing ?

stategy: from the hard drive: go through Diego's data and write all the .Table files in a csv.
then read the csv here, extract task and subject id from the csv 
BE CAREFUL about the OP data as the date matters 
then merge the diego to test 
write test into csv 


In [15]:
file = open(r"/Users/denizjafari/Documents/CODE/ClinicalScore/ClinicalScore/parsed_data_diego.txt","r") 

print(file.read())

./ALS/new/AF13/A013_02_BBP_NORMAL_video.Table
./ALS/new/AF13/A013_02_NSM_BIGSMILE_video.Table
./ALS/new/AF13/A013_02_NSM_OPEN_video.Table
./ALS/new/AF13/A013_02_NSM_SPREAD_video.Table
./ALS/new/AF14/A014_02_BBP_NORMAL_video.Table
./ALS/new/AF14/A014_02_NSM_BIGSMILE_video.Table
./ALS/new/AF14/A014_02_NSM_OPEN_video.Table
./ALS/new/AF14/A014_02_NSM_SPREAD_video.Table
./ALS/new/AF16/A016_02_BBP_NORMAL_video.Table
./ALS/new/AF16/A016_02_NSM_BIGSMILE_video.Table
./ALS/new/AF16/A016_02_NSM_OPEN_video.Table
./ALS/new/AF16/A016_02_NSM_SPREAD_video.Table
./ALS/new/AF16/A016_02_RST_REST3_video.Table
./ALS/new/AF17/A017_02_BBP_NORMAL_video.Table
./ALS/new/AF17/A017_02_NSM_BIGSMILE_video.Table
./ALS/new/AF17/A017_02_NSM_OPEN_video.Table
./ALS/new/AF17/A017_02_NSM_SPREAD_video.Table
./ALS/new/AF17/A017_02_RST_REST3_video.Table
./ALS/new/AF18/AF18_02_NSM_BIGSMILE_video.Table
./ALS/new/AF18/AF18_02_NSM_OPEN_video.Table
./ALS/new/AF18/AF18_02_NSM_SPREAD_video.Table
./ALS/new/AF18/AF18_02_RST_REST_vide

In [16]:
parsed = pd.read_csv("/Users/denizjafari/Documents/CODE/ClinicalScore/ClinicalScore/parsed_data_diego.txt",  header=None)
print(parsed.shape)
parsed

(236, 1)


,0
0,./ALS/new/AF13/A013_02_BBP_NORMAL_video.Table
1,./ALS/new/AF13/A013_02_NSM_BIGSMILE_video.Table
2,./ALS/new/AF13/A013_02_NSM_OPEN_video.Table
3,./ALS/new/AF13/A013_02_NSM_SPREAD_video.Table
4,./ALS/new/AF14/A014_02_BBP_NORMAL_video.Table
...,...
231,./Post-Stroke/old/S012/S012_02_NSM_OPEN_video....
232,./Post-Stroke/old/S012/S012_02_NSM_SPREAD_vide...
233,./Post-Stroke/old/S013/S013_02_BBP_NORMAL_audi...
234,./Post-Stroke/old/S013/S013_02_NSM_OPEN_video....


In [17]:
parsed.rename(columns = {0: 'file_path'}, inplace = True)

parsed['file_name'] = [os.path.basename(x) for x in parsed['file_path']]

parsed['subject_id'] = [get_subject_id(x) for x in parsed['file_name']]
parsed['task'] = [set_task(x) for x in parsed['file_name']]
parsed['Need_Parsing'] = np.where(parsed['file_name'].isnull(), 1, 0)



In [18]:
parsed['subject_id'] = [id_label_change(x) for x in parsed['subject_id']]
parsed

,file_path,file_name,subject_id,task,Need_Parsing
0,./ALS/new/AF13/A013_02_BBP_NORMAL_video.Table,A013_02_BBP_NORMAL_video.Table,AF13,BBP_NORMAL,0
1,./ALS/new/AF13/A013_02_NSM_BIGSMILE_video.Table,A013_02_NSM_BIGSMILE_video.Table,AF13,NSM_BIGSMILE,0
2,./ALS/new/AF13/A013_02_NSM_OPEN_video.Table,A013_02_NSM_OPEN_video.Table,AF13,NSM_OPEN,0
3,./ALS/new/AF13/A013_02_NSM_SPREAD_video.Table,A013_02_NSM_SPREAD_video.Table,AF13,NSM_SPREAD,0
4,./ALS/new/AF14/A014_02_BBP_NORMAL_video.Table,A014_02_BBP_NORMAL_video.Table,AF14,BBP_NORMAL,0
...,...,...,...,...,...
231,./Post-Stroke/old/S012/S012_02_NSM_OPEN_video....,S012_02_NSM_OPEN_video.Table,S012,NSM_OPEN,0
232,./Post-Stroke/old/S012/S012_02_NSM_SPREAD_vide...,S012_02_NSM_SPREAD_video.Table,S012,NSM_SPREAD,0
233,./Post-Stroke/old/S013/S013_02_BBP_NORMAL_audi...,S013_02_BBP_NORMAL_audio.Table,S013,BBP_NORMAL,0
234,./Post-Stroke/old/S013/S013_02_NSM_OPEN_video....,S013_02_NSM_OPEN_video.Table,S013,NSM_OPEN,0


In [19]:
test2 = test.merge(parsed, how= 'left', on=['subject_id', 'task'])
test2.loc[test2['Need_Parsing'].isnull(), 'Need_Parsing'] = 1
test2

,file_path_x,file_name_x,subject_id,task,Need_Clinical_Score,file_path_y,file_name_y,Need_Parsing
0,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_BBP_N...,AF01_02_BBP_NORMAL.rssdk,AF01,BBP_NORMAL,1,./ALS/old/A001/A001_02_BBP_NORMAL_audio.Table,A001_02_BBP_NORMAL_audio.Table,0.0
1,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PA.rssdk,AF01,DDK_PA,1,NaN,NaN,1.0
2,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_DDK_P...,AF01_02_DDK_PATAKA.rssdk,AF01,DDK_PA,1,NaN,NaN,1.0
3,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_K...,AF01_02_NSM_KISS.rssdk,AF01,NSM_KISS,1,NaN,NaN,1.0
4,./ALS/AF01_SBK_20170509/02/RSSDK/AF01_02_NSM_O...,AF01_02_NSM_OPEN.rssdk,AF01,NSM_OPEN,1,./ALS/old/A001/A001_02_NSM_OPEN_video.Table,A001_02_NSM_OPEN_video.Table,0.0
...,...,...,...,...,...,...,...,...
1240,./Post-Stroke/S013_TRI_20180302/02/RSSDK/S013_...,S013_02_RST_REST3.rssdk,S013,RST_REST,1,NaN,NaN,1.0
1241,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_KISS.rssdk,S004,NSM_KISS,1,NaN,NaN,1.0
1242,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_OPEN.rssdk,S004,NSM_OPEN,1,NaN,NaN,1.0
1243,./Post-Stroke/_S004_TRI_20170808/02/S004_02_NS...,S004_02_NSM_SPREAD.rssdk,S004,NSM_SPREAD,1,NaN,NaN,1.0


In [20]:
test2 = test2.drop(['file_path_y', 'file_name_y'],axis=1)



In [21]:
ids = ['A','K', 'OP', 'S', 'N', 'PD', 'P']

def pop_labeler(row):
    for i in ids:
        if i in row:
            if i == 'OP' or i == "S":         
                return 'Stroke'
            elif i == "A" or i == "P":
                return 'ALS'
            elif i == "PD":
                return 'PD'
            else:
                return 'HC'
            
test2['population'] = [pop_labeler(x) for x in test2['subject_id']]

In [22]:
test2.groupby(['population', 'Need_Clinical_Score']).count()

file_path_x  file_name_x  subject_id  task  \
population Need_Clinical_Score                                               
ALS        0                            119          119         119   119   
           1                            158          158         158   158   
HC         0                            145          145         145   145   
           1                            258          258         258   258   
PD         1                             91           91          91    91   
Stroke     0                            177          177         177   177   
           1                            289          289         289   289   

                                Need_Parsing  
population Need_Clinical_Score                
ALS        0                             119  
           1                             158  
HC         0                             145  
           1                             258  
PD         1                              91  
Stroke     0                             177  
           1                             289

In [23]:
test2.groupby(['population', 'Need_Parsing']).count()

file_path_x  file_name_x  subject_id  task  \
population Need_Parsing                                               
ALS        0.0                    89           89          89    89   
           1.0                   188          188         188   188   
HC         0.0                   140          140         140   140   
           1.0                   263          263         263   263   
PD         1.0                    91           91          91    91   
Stroke     0.0                   247          247         247   247   
           1.0                   219          219         219   219   

                         Need_Clinical_Score  
population Need_Parsing                       
ALS        0.0                            89  
           1.0                           188  
HC         0.0                           140  
           1.0                           263  
PD         1.0                            91  
Stroke     0.0                           247  
           1.0                           219

In [24]:
test2.groupby(['subject_id']).count()

,file_path_x,file_name_x,task,Need_Clinical_Score,Need_Parsing,population
subject_id,,,,,,
001,4,4,4,4,4,0
002,4,4,4,4,4,0
AF01,14,14,14,14,14,14
AF02,18,18,18,18,18,18
AF03,14,14,14,14,14,14
...,...,...,...,...,...,...
S009,9,9,9,9,9,9
S010,7,7,7,7,7,7
S011,11,11,11,11,11,11


In [32]:
test2.groupby(['population', 'task', 'Need_Parsing']).count()

file_path_x  file_name_x  subject_id  \
population task         Need_Parsing                                         
ALS        BBP_NORMAL   0.0                    23           23          23   
                        1.0                     8            8           8   
           DDK_PA       1.0                    93           93          93   
           NSM_BIGSMILE 0.0                     7            7           7   
                        1.0                     4            4           4   
           NSM_KISS     1.0                    28           28          28   
           NSM_OPEN     0.0                    25           25          25   
                        1.0                     8            8           8   
           NSM_SPREAD   0.0                    25           25          25   
                        1.0                     7            7           7   
           RST_REST     0.0                     9            9           9   
                        1.0                    40           40          40   
HC         BBP_NORMAL   0.0                    34           34          34   
                        1.0                    12           12          12   
           DDK_PA       1.0                   129          129         129   
           NSM_BIGSMILE 0.0                    18           18          18   
                        1.0                     4            4           4   
           NSM_KISS     1.0                    28           28          28   
           NSM_OPEN     0.0                    41           41          41   
                        1.0                     6            6           6   
           NSM_SPREAD   0.0                    41           41          41   
                        1.0                     6            6           6   
           RST_REST     0.0                     6            6           6   
                        1.0                    78           78          78   
PD         BBP_NORMAL   1.0                    11           11          11   
           DDK_PA       1.0                    22           22          22   
           NSM_BIGSMILE 1.0                    11           11          11   
           NSM_OPEN     1.0                    12           12          12   
           NSM_SPREAD   1.0                    11           11          11   
           RST_REST     1.0                    24           24          24   
Stroke     BBP_NORMAL   0.0                    42           42          42   
                        1.0                     3            3           3   
           DDK_PA       1.0                   151          151         151   
           NSM_BIGSMILE 0.0                    30           30          30   
                        1.0                     5            5           5   
           NSM_KISS     1.0                    20           20          20   
           NSM_OPEN     0.0                    42           42          42   
                        1.0                     5            5           5   
           NSM_SPREAD   0.0                    43           43          43   
                        1.0                     5            5           5   
           RST_REST     0.0                    90           90          90   
                        1.0                    30           30          30   

                                      Need_Clinical_Score  
population task         Need_Parsing                       
ALS        BBP_NORMAL   0.0                            23  
                        1.0                             8  
           DDK_PA       1.0                            93  
           NSM_BIGSMILE 0.0                             7  
                        1.0                             4  
           NSM_KISS     1.0                            28  
           NSM_OPEN     0.0                            25  
                        1.0                             8  
     

In [36]:
print(test2.subject_id.unique())
print(len(test2.subject_id.unique()))

['AF01' 'AF02' 'AF03' 'AF04' 'AF05' 'AF06' 'AF07' 'AF09' 'AF10' 'AF11'
 'AF12' 'AF13' 'AF14' 'AF16' 'AF17' 'AF18' 'AF19' 'ASH1' 'NF00' 'NF01'
 'NF02' 'NF03' 'NF04' 'NF05' 'NF06' 'NF07' 'NF08' 'NF09' 'NF10' 'NF11'
 'NF12' 'NF13' 'NF14' 'NF15' 'NF16' 'NF17' 'NF18' 'NF19' 'NF20' 'NF21'
 'NF22' 'NF23' 'NF24' 'NF25' 'NF26' 'NF27' '001' '002' 'K001' 'K003'
 'PD01' 'PD02' 'PD03' 'PD04' 'PD05' 'PD06' 'PD07' 'PD08' 'P002' 'P003'
 'P004' 'OP01' 'OP02' 'OP03' 'OP04' 'OP05' 'OP06' 'OP07' 'OP08' 'OP09'
 'OP10' 'OP11' 'OP12' 'OP13' 'OP14' 'S001' 'S002' 'S003' 'S005' 'S006'
 'S007' 'S008' 'S009' 'S010' 'S011' 'S012' 'S013' 'S004']
88


In [25]:
csv_name = 'clinical_score_dataset2.csv'
test2.to_csv(csv_name)

## Testing random 

In [26]:
print(data['file_name'][2])
data['file_name'][2].split('_')[2:4]

AF01_02_BBP_NORMAL.rssdk


['BBP', 'NORMAL.rssdk']

In [27]:
#data = data[~data['subject_id'].isin(['001', '002', 'K001', 'K003', 'PD01', 'PD02', 'PD03', 'PD04',
#       'PD05', 'PD06', 'PD07', 'PD08'])]

data['Need_Clinical_Score'] = data[~data['subject_id'].isin(['001', '002', 'K001', 'K003', 'PD01', 'PD02', 'PD03', 'PD04',
       'PD05', 'PD06', 'PD07', 'PD08'])]


data['subject_id'].unique()

ValueError: Wrong number of items passed 4, placement implies 1